In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from uszipcode import SearchEngine
search = SearchEngine()

import utils as u
from config.GLOBAL import *

C:\Users\steff\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
census_tracts_2010_filtered = census_tracts_2010
census_tracts_2010_filtered = [x for x in census_tracts_2010_filtered if x not in census_tracts_2010_0n_veh]
census_tracts_2010_filtered = [x for x in census_tracts_2010_filtered if x not in census_tracts_2010_less_than_100n_veh]
census_tracts_2010_filtered = [x for x in census_tracts_2010_filtered if x not in census_tracts_2010_missing_ACS_data]
census_tracts_2010_filtered = [x for x in census_tracts_2010_filtered if x not in census_tracts_2010_]

In [29]:
# multiindex = pd.MultiIndex.from_product([times, census_tracts_2010], names=["time", "census_tract_2010"])
multiindex = pd.MultiIndex.from_product([times_future, census_tracts_2010_filtered], names=["time", "census_tract_2010"])
# multiindex = pd.MultiIndex.from_product([times_all, census_tracts_2010_filtered], names=["date", "census_tract_2010"])

In [30]:
# create final dataframe and add county and ZIP code column to it
df_future = pd.DataFrame(index=multiindex, columns=["county", "zip_code"])
# for census_tract in census_tracts_2010:
for census_tract in census_tracts_2010_filtered:
    if census_tract in tract_to_zip.index:
        zip_code = tract_to_zip.loc[census_tract, "zip_code"]
        df_future.loc[(slice(None), census_tract), "zip_code"] = zip_code
        df_future.loc[(slice(None), census_tract), "county"] = search.by_zipcode(zip_code).county.replace(" County", "")
df_future

county zip_code
time       census_tract_2010                 
2022-10-31 53001950100         Adams    99169
           53001950200         Adams    99371
           53001950300         Adams    99344
           53001950400         Adams    99344
           53001950500         Adams    99344
...                              ...      ...
2035-12-31 53077940002        Yakima    98948
           53077940003        Yakima    98903
           53077940004        Yakima    98951
           53077940005        Yakima    98948
           53077940006        Yakima    98948

[228483 rows x 2 columns]

In [31]:
empty_df_future = df_future.copy()

In [60]:
df = empty_df_future.copy()

In [23]:
#read location fixed effects
fixeff_i = pd.read_csv("results/model~1/model_fixeff_i.csv")
fixeff_i = fixeff_i.rename({"x": "fixeff_i"}, axis=1)
fixeff_i = fixeff_i.set_index("census_tract_2010")
fixeff_i

,fixeff_i
census_tract_2010,
53001950100,0.000000
53001950200,-2.905202
53001950300,-0.418235
53001950400,-2.304883
53001950500,-1.290576
...,...
53077940002,-1.583640
53077940003,-1.470011
53077940004,-2.144669


In [7]:
#read time fixed effects
# fixef_t = pd.read_csv("results/model~1/model_fixef_t.csv")
fixeff_t = pd.read_csv("results/model~1/model_fixeff_t_predict.csv")
fixeff_t = fixeff_t.rename({"date_predict": "date"}, axis=1)
fixeff_t = fixeff_t.drop("Unnamed: 0", axis=1)
# fixeff_t = fixeff_t.convert_dtypes()
fixeff_t["date"] = pd.to_datetime(fixeff_t["date"]).dt.date
fixeff_t = fixeff_t.set_index("date")
fixeff_t

,t_dummy_predict,fixeff_t_predict
date,,
2011-01-31,1,-0.583585
2011-02-28,2,-0.253416
2011-03-31,3,0.044707
2011-04-30,4,0.316458
2011-05-31,5,0.566126
...,...,...
2035-08-31,296,10.297228
2035-09-30,297,10.307330
2035-10-31,298,10.317398


In [19]:
df_past = pd.read_csv("data/data_2011_to_2022_R.csv", parse_dates=["time", "date"])
df_past["date"] = pd.to_datetime(df_past["date"]).dt.date
df_past = df_past.sort_values("index")
df_past = df_past.set_index(["date", "census_tract_2010"])
df_past

Unnamed: 0       time  county  \
date       census_tract_2010                                              
2011-01-31 53001950100        53001950100-2011-01-31 2011-01-31   Adams   
           53001950200        53001950200-2011-01-31 2011-01-31   Adams   
           53001950300        53001950300-2011-01-31 2011-01-31   Adams   
           53001950400        53001950400-2011-01-31 2011-01-31   Adams   
           53001950500        53001950500-2011-01-31 2011-01-31   Adams   
...                                              ...        ...     ...   
2022-09-30 53077940002        53077940002-2022-09-30 2022-09-30  Yakima   
           53077940003        53077940003-2022-09-30 2022-09-30  Yakima   
           53077940004        53077940004-2022-09-30 2022-09-30  Yakima   
           53077940005        53077940005-2022-09-30 2022-09-30  Yakima   
           53077940006        53077940006-2022-09-30 2022-09-30  Yakima   

                              zip_code  n_ev  n_bev  n_phev  m_ev  m_bev  \
date       census_tract_2010                                               
2011-01-31 53001950100           99169  0.01   0.01    0.01     7      6   
           53001950200           99371  0.01   0.01    0.01     7      6   
           53001950300           99344  0.01   0.01    0.01     7      6   
           53001950400           99344  0.01   0.01    0.01     7      6   
           53001950500           99344  0.01   0.01    0.01     7      6   
...                                ...   ...    ...     ...   ...    ...   
2022-09-30 53077940002           98948  6.00   4.00    2.00   121     60   
           53077940003           98903  4.00   1.00    3.00   121     60   
           53077940004           98951  2.00   2.00    0.01   121     60   
           53077940005           98948  1.00   1.00    0.01   121     60   
           53077940006           98948  2.00   2.00    0.01   121     60   

                              m_phev  ...   p_white  p_bachelor  \
date       census_tract_2010          ...                         
2011-01-31 53001950100             1  ...  0.930305    0.146269   
           53001950200             1  ...  0.842457    0.320513   
           53001950300             1  ...  0.503580    0.062859   
           53001950400             1  ...  0.644241    0.148916   
           53001950500             1  ...  0.516711    0.061387   
...                              ...  ...       ...         ...   
2022-09-30 53077940002            64  ...  0.438808    0.100344   
           53077940003            64  ...  0.462168    0.239831   
           53077940004            64  ...  0.537067    0.046347   
           53077940005            64  ...  0.601439    0.086981   
           53077940006            64  ...  0.549848    0.039939   

                              p_single_family_unit              tract_date  \
date       census_tract_2010                                                 
2011-01-31 53001950100                    0.813604  53001950100-2011-01-31   
           53001950200                    0.792271  53001950200-2011-01-31   
           53001950300                    0.348436  53001950300-2011-01-31   
           53001950400                    0.622989  53001950400-2011-01-31   
           53001950500                    0.726737  53001950500-2011-01-31   
...                                            ...                     ...   
2022-09-30 53077940002                    0.823129  53077940002-2022-09-30   
           53077940003                    0.698741  53077940003-2022-09-30   
           53077940004                    0.688690  53077940004-2022-09-30   
           53077940005                    0.750708  53077940005-2022-09-30   
           53077940006                    0.795062  53077940006-2022-09-30   

                               index  index_p  p_ev_log_odds_res  \
date       census_tract_2010                                       
2011-01-31 53001950100             1        1         

In [32]:
df = pd.concat([df_past,df_future])
df

Unnamed: 0       time  county  \
date       census_tract_2010                                              
2011-01-31 53001950100        53001950100-2011-01-31 2011-01-31   Adams   
           53001950200        53001950200-2011-01-31 2011-01-31   Adams   
           53001950300        53001950300-2011-01-31 2011-01-31   Adams   
           53001950400        53001950400-2011-01-31 2011-01-31   Adams   
           53001950500        53001950500-2011-01-31 2011-01-31   Adams   
...                                              ...        ...     ...   
2035-12-31 53077940002                           NaN        NaT  Yakima   
           53077940003                           NaN        NaT  Yakima   
           53077940004                           NaN        NaT  Yakima   
           53077940005                           NaN        NaT  Yakima   
           53077940006                           NaN        NaT  Yakima   

                             zip_code  n_ev  n_bev  n_phev  m_ev  m_bev  \
date       census_tract_2010                                              
2011-01-31 53001950100          99169  0.01   0.01    0.01   7.0    6.0   
           53001950200          99371  0.01   0.01    0.01   7.0    6.0   
           53001950300          99344  0.01   0.01    0.01   7.0    6.0   
           53001950400          99344  0.01   0.01    0.01   7.0    6.0   
           53001950500          99344  0.01   0.01    0.01   7.0    6.0   
...                               ...   ...    ...     ...   ...    ...   
2035-12-31 53077940002          98948   NaN    NaN     NaN   NaN    NaN   
           53077940003          98903   NaN    NaN     NaN   NaN    NaN   
           53077940004          98951   NaN    NaN     NaN   NaN    NaN   
           53077940005          98948   NaN    NaN     NaN   NaN    NaN   
           53077940006          98948   NaN    NaN     NaN   NaN    NaN   

                              m_phev  ...   p_white  p_bachelor  \
date       census_tract_2010          ...                         
2011-01-31 53001950100           1.0  ...  0.930305    0.146269   
           53001950200           1.0  ...  0.842457    0.320513   
           53001950300           1.0  ...  0.503580    0.062859   
           53001950400           1.0  ...  0.644241    0.148916   
           53001950500           1.0  ...  0.516711    0.061387   
...                              ...  ...       ...         ...   
2035-12-31 53077940002           NaN  ...       NaN         NaN   
           53077940003           NaN  ...       NaN         NaN   
           53077940004           NaN  ...       NaN         NaN   
           53077940005           NaN  ...       NaN         NaN   
           53077940006           NaN  ...       NaN         NaN   

                              p_single_family_unit              tract_date  \
date       census_tract_2010                                                 
2011-01-31 53001950100                    0.813604  53001950100-2011-01-31   
           53001950200                    0.792271  53001950200-2011-01-31   
           53001950300                    0.348436  53001950300-2011-01-31   
           53001950400                    0.622989  53001950400-2011-01-31   
           53001950500                    0.726737  53001950500-2011-01-31   
...                                            ...                     ...   
2035-12-31 53077940002                         NaN                     NaN   
           53077940003                         NaN                     NaN   
           53077940004                         NaN                     NaN   
           53077940005                         NaN                     NaN   
           53077940006                         NaN                     NaN   

                              index  index_p  p_ev_log_odds_res  \
date       census_tract_2010                                      
2011-01-31 53001950100          1.0      1.0          -1.891626   
  

In [33]:
df_copy = df.copy()

In [24]:
df = df_copy

In [48]:
# n_veh = data.loc[data["date"] == pd.to_datetime("2022-09-30").date(), ["date", "n_veh"]]
# n_veh = n_veh.set_index("date")
# n_veh

,n_veh
date,
2022-09-30,2188
2022-09-30,1477
2022-09-30,6293
2022-09-30,2103
2022-09-30,4260
...,...
2022-09-30,3548
2022-09-30,2279
2022-09-30,4483


In [68]:
# df = empty_df.copy()

In [34]:
df = df.join(fixeff_i, on=["census_tract_2010"], how="left")
df = df.join(fixeff_t, on=["date"], how="left")
# df = df.join(data, on=["date"], how="left")
df

Unnamed: 0       time  county  \
date       census_tract_2010                                              
2011-01-31 53001950100        53001950100-2011-01-31 2011-01-31   Adams   
           53001950200        53001950200-2011-01-31 2011-01-31   Adams   
           53001950300        53001950300-2011-01-31 2011-01-31   Adams   
           53001950400        53001950400-2011-01-31 2011-01-31   Adams   
           53001950500        53001950500-2011-01-31 2011-01-31   Adams   
...                                              ...        ...     ...   
2035-12-31 53077940002                           NaN        NaT  Yakima   
           53077940003                           NaN        NaT  Yakima   
           53077940004                           NaN        NaT  Yakima   
           53077940005                           NaN        NaT  Yakima   
           53077940006                           NaN        NaT  Yakima   

                             zip_code  n_ev  n_bev  n_phev  m_ev  m_bev  \
date       census_tract_2010                                              
2011-01-31 53001950100          99169  0.01   0.01    0.01   7.0    6.0   
           53001950200          99371  0.01   0.01    0.01   7.0    6.0   
           53001950300          99344  0.01   0.01    0.01   7.0    6.0   
           53001950400          99344  0.01   0.01    0.01   7.0    6.0   
           53001950500          99344  0.01   0.01    0.01   7.0    6.0   
...                               ...   ...    ...     ...   ...    ...   
2035-12-31 53077940002          98948   NaN    NaN     NaN   NaN    NaN   
           53077940003          98903   NaN    NaN     NaN   NaN    NaN   
           53077940004          98951   NaN    NaN     NaN   NaN    NaN   
           53077940005          98948   NaN    NaN     NaN   NaN    NaN   
           53077940006          98948   NaN    NaN     NaN   NaN    NaN   

                              m_phev  ...              tract_date  index  \
date       census_tract_2010          ...                                  
2011-01-31 53001950100           1.0  ...  53001950100-2011-01-31    1.0   
           53001950200           1.0  ...  53001950200-2011-01-31    2.0   
           53001950300           1.0  ...  53001950300-2011-01-31    3.0   
           53001950400           1.0  ...  53001950400-2011-01-31    4.0   
           53001950500           1.0  ...  53001950500-2011-01-31    5.0   
...                              ...  ...                     ...    ...   
2035-12-31 53077940002           NaN  ...                     NaN    NaN   
           53077940003           NaN  ...                     NaN    NaN   
           53077940004           NaN  ...                     NaN    NaN   
           53077940005           NaN  ...                     NaN    NaN   
           53077940006           NaN  ...                     NaN    NaN   

                              index_p  p_ev_log_odds_res  \
date       census_tract_2010                               
2011-01-31 53001950100            1.0          -1.891626   
           53001950200          142.0          -5.066795   
           53001950300          283.0          -1.253086   
           53001950400          424.0          -4.198314   
           53001950500          565.0          -2.377608   
...                               ...                ...   
2035-12-31 53077940002            NaN                NaN   
           53077940003            NaN                NaN   
           53077940004            NaN                NaN   
           53077940005            NaN                NaN   
           53077940006            NaN                NaN   

                              p_ev_log_odds_predict  p_ev_predict  p_ev_res  \
date       census_tract_2010                                                  
2011-01-31 53001950100                   -14.130161  7.300438e-07 -0.000004   
           53001950200                   -16.943662  4.379870e-08 -0.000007   
    

In [ ]:
#Fill in forecasts of predictors and n_veh

In [36]:
# df["n_veh"] = 0
# df.loc[(times, slice(None)), "n_veh"] = data.loc[(times, slice(None)), "n_veh"]
for time in times_future:
    print(time, "", end="")
    # print(df.loc[(time, slice(None)), "n_veh"])
    # print(data.loc[(times[-1], slice(None)), "n_veh"])
    # print(len(df.loc[(time, slice(None)), "n_veh"]))
    # print(len(df_past.loc[(times[-1], slice(None)), "n_veh"].values))
    # i
    df.loc[(time, slice(None)), "n_veh"] = df_past.loc[(times[-1], slice(None)), "n_veh"].values
df

2022-10-31 2022-11-30 2022-12-31 2023-01-31 2023-02-28 2023-03-31 2023-04-30 2023-05-31 2023-06-30 2023-07-31 2023-08-31 2023-09-30 2023-10-31 2023-11-30 2023-12-31 2024-01-31 2024-02-29 2024-03-31 2024-04-30 2024-05-31 2024-06-30 2024-07-31 2024-08-31 2024-09-30 2024-10-31 2024-11-30 2024-12-31 2025-01-31 2025-02-28 2025-03-31 2025-04-30 2025-05-31 2025-06-30 2025-07-31 2025-08-31 2025-09-30 2025-10-31 2025-11-30 2025-12-31 2026-01-31 2026-02-28 2026-03-31 2026-04-30 2026-05-31 2026-06-30 2026-07-31 2026-08-31 2026-09-30 2026-10-31 2026-11-30 2026-12-31 2027-01-31 2027-02-28 2027-03-31 2027-04-30 2027-05-31 2027-06-30 2027-07-31 2027-08-31 2027-09-30 2027-10-31 2027-11-30 2027-12-31 2028-01-31 2028-02-29 2028-03-31 2028-04-30 2028-05-31 2028-06-30 2028-07-31 2028-08-31 2028-09-30 2028-10-31 2028-11-30 2028-12-31 2029-01-31 2029-02-28 2029-03-31 2029-04-30 2029-05-31 2029-06-30 2029-07-31 2029-08-31 2029-09-30 2029-10-31 2029-11-30 2029-12-31 2030-01-31 2030-02-28 2030-03-31 2030-04-30

Unnamed: 0       time  county  \
date       census_tract_2010                                              
2011-01-31 53001950100        53001950100-2011-01-31 2011-01-31   Adams   
           53001950200        53001950200-2011-01-31 2011-01-31   Adams   
           53001950300        53001950300-2011-01-31 2011-01-31   Adams   
           53001950400        53001950400-2011-01-31 2011-01-31   Adams   
           53001950500        53001950500-2011-01-31 2011-01-31   Adams   
...                                              ...        ...     ...   
2035-12-31 53077940002                           NaN        NaT  Yakima   
           53077940003                           NaN        NaT  Yakima   
           53077940004                           NaN        NaT  Yakima   
           53077940005                           NaN        NaT  Yakima   
           53077940006                           NaN        NaT  Yakima   

                             zip_code  n_ev  n_bev  n_phev  m_ev  m_bev  \
date       census_tract_2010                                              
2011-01-31 53001950100          99169  0.01   0.01    0.01   7.0    6.0   
           53001950200          99371  0.01   0.01    0.01   7.0    6.0   
           53001950300          99344  0.01   0.01    0.01   7.0    6.0   
           53001950400          99344  0.01   0.01    0.01   7.0    6.0   
           53001950500          99344  0.01   0.01    0.01   7.0    6.0   
...                               ...   ...    ...     ...   ...    ...   
2035-12-31 53077940002          98948   NaN    NaN     NaN   NaN    NaN   
           53077940003          98903   NaN    NaN     NaN   NaN    NaN   
           53077940004          98951   NaN    NaN     NaN   NaN    NaN   
           53077940005          98948   NaN    NaN     NaN   NaN    NaN   
           53077940006          98948   NaN    NaN     NaN   NaN    NaN   

                              m_phev  ...              tract_date  index  \
date       census_tract_2010          ...                                  
2011-01-31 53001950100           1.0  ...  53001950100-2011-01-31    1.0   
           53001950200           1.0  ...  53001950200-2011-01-31    2.0   
           53001950300           1.0  ...  53001950300-2011-01-31    3.0   
           53001950400           1.0  ...  53001950400-2011-01-31    4.0   
           53001950500           1.0  ...  53001950500-2011-01-31    5.0   
...                              ...  ...                     ...    ...   
2035-12-31 53077940002           NaN  ...                     NaN    NaN   
           53077940003           NaN  ...                     NaN    NaN   
           53077940004           NaN  ...                     NaN    NaN   
           53077940005           NaN  ...                     NaN    NaN   
           53077940006           NaN  ...                     NaN    NaN   

                              index_p  p_ev_log_odds_res  \
date       census_tract_2010                               
2011-01-31 53001950100            1.0          -1.891626   
           53001950200          142.0          -5.066795   
           53001950300          283.0          -1.253086   
           53001950400          424.0          -4.198314   
           53001950500          565.0          -2.377608   
...                               ...                ...   
2035-12-31 53077940002            NaN                NaN   
           53077940003            NaN                NaN   
           53077940004            NaN                NaN   
           53077940005            NaN                NaN   
           53077940006            NaN                NaN   

                              p_ev_log_odds_predict  p_ev_predict  p_ev_res  \
date       census_tract_2010                                                  
2011-01-31 53001950100                   -14.130161  7.300438e-07 -0.000004   
           53001950200                   -16.943662  4.379870e-08 -0.000007   
    

In [42]:
df.to_csv("data/data_future.csv")